<a href="https://colab.research.google.com/github/iherdt/airbyte/blob/main/pyairbyte_notebooks/PyAirbyte_Basic_Features_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyAirbyte Demo

Below is a pre-release demo of PyAirbyte.


## Install PyAirbyte


In [13]:
# Add virtual environment support for running in Google Colab
!apt-get install -qq python3.10-venv

# Install PyAirbyte
%pip install --quiet airbyte

## Locating your Data Source

To see what data sources are available, you can check [our docs](https://docs.airbyte.com/using-airbyte/airbyte-lib/getting-started) or run the following:


In [14]:
# Import PyAirbyte
import airbyte as ab

# Show all available connectors
ab.get_available_connectors()

['destination-astra',
 'destination-aws-datalake',
 'destination-chroma',
 'destination-cobra',
 'destination-convex',
 'destination-deepset',
 'destination-duckdb',
 'destination-firebolt',
 'destination-firestore',
 'destination-glassflow',
 'destination-google-sheets',
 'destination-milvus',
 'destination-motherduck',
 'destination-pgvector',
 'destination-pinecone',
 'destination-qdrant',
 'destination-rabbitmq',
 'destination-sftp-json',
 'destination-snowflake-cortex',
 'destination-sqlite',
 'destination-timeplus',
 'destination-typesense',
 'destination-vectara',
 'destination-weaviate',
 'source-7shifts',
 'source-activecampaign',
 'source-adjust',
 'source-agilecrm',
 'source-aha',
 'source-airbyte',
 'source-aircall',
 'source-airtable',
 'source-akeneo',
 'source-algolia',
 'source-alpha-vantage',
 'source-amazon-ads',
 'source-amazon-seller-partner',
 'source-amazon-sqs',
 'source-amplitude',
 'source-apify-dataset',
 'source-appcues',
 'source-appfigures',
 'source-appfol

## Load the Source Data using PyAirbyte


Create and install a source connector:


In [15]:
import airbyte as ab

# Create and install the source:
source: ab.Source = ab.get_source("source-faker")

Writing `source-faker` logs to file: /tmp/airbyte/logs/source-faker/source-faker-log-JQHWMEWK1.log


In [16]:
# Configure the source
source.set_config(
    config={
        "count": 50_000,  # Adjust this to get a larger or smaller dataset
        "seed": 123,
    },
)
# Verify the config and creds by running `check`:
source.check()

Connection check succeeded for `source-faker`.

## Read Data from the PyAirbyte Cache

Once data is read, we can do anything we want to with the resulting streams. This includes `to_pandas()` which registers a Pandas dataframe and `to_sql_table()` which gives us a SQLAlchemy `Table` boject, which we can use to run SQL queries.


In [ ]:
# Select all of the source's streams and read data into the internal cache:
source.select_all_streams()
read_result: ab.ReadResult = source.read()

In [ ]:
# Display or transform the loaded data
products_df = read_result["products"].to_pandas()
display(products_df)

## Creating graphs

PyAirbyte integrates with Pandas, which integrates with `matplotlib` as well as many other popular libraries. We can use this as a means of quickly creating graphs.


In [ ]:
%pip install matplotlib

import matplotlib.pyplot as plt

users_df = read_result["users"].to_pandas()

plt.hist(users_df["age"], bins=10, edgecolor="black")
plt.title("Histogram of Ages")
plt.xlabel("Ages")
plt.ylabel("Frequency")
plt.show()

## Working in SQL

Since data is cached in a local DuckDB database, we can query the data with SQL.

We can do this in multiple ways. One way is to use the [JupySQL Extension](https://jupysql.ploomber.io/en/latest/user-guide/template.html), which we'll use below.


In [ ]:
# Install JupySQL to enable SQL cell magics
%pip install --quiet jupysql
# Load JupySQL extension
%load_ext sql
# Configure max row limit (optional)
%config SqlMagic.displaylimit = 200

In [ ]:
# Get the SQLAlchemy 'engine' object for the cache
engine = read_result.cache.get_sql_engine()
# Pass the engine to JupySQL
%sql engine

In [ ]:
# Get table objects for the 'users' and 'purchases' streams
users_table = read_result.cache["users"].to_sql_table()
purchases_table = read_result.cache["purchases"].to_sql_table()
display([users_table.fullname, purchases_table.fullname])

In [ ]:
%%sql
# Show most recent purchases by purchase date:
SELECT users.id, users.name, purchases.product_id, purchases.purchased_at
FROM {{ users_table.fullname }} AS users
JOIN {{ purchases_table.fullname }} AS purchases
ON users.id = purchases.user_id
ORDER BY purchases.purchased_at DESC
LIMIT 10

In [ ]:
# Show tables for the other streams
%sqlcmd tables